In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime, timedelta
import csv

In [1]:
print("Hello World")

Hello World


## Objective:
#### -Import the datasets. 
#### -Create algorithm that searches for matches that are +/- 10 second apart and remove the rest (maybe two seconds)
#### -Do spectral analysis on the data (counts/bin)

## Importing data

In [ ]:
#Importing and redefining the dataframes
#Not loading the trigger list for 2021 yet

ipn_data = pd.read_csv("trigIPN.csv", sep="|")
ipn = pd.DataFrame(ipn_data)
ipn.drop(columns= ['Unnamed: 0', 'Unnamed: 2'], axis=1, inplace=True) #dropping unwanted columns
ipn.rename(columns={ipn.columns[0]:"time"}, inplace = True)

trigB_data = pd.read_csv("./ASIM/trigB.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
trigB = pd.DataFrame(trigB_data)
trigB.drop(columns=["######"], inplace=True) #dropping unwanted columns
trigB.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", "Corr":"corr"}, inplace=True) #renaming columns

#trigB_21_data = pd.read_csv("./ASIM/trigB_2021.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
#trigB_21 = pd.DataFrame(trigB_21_data)
#trigB_21.drop(columns=["######"], inplace=True)
#trigB_21.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time" }, inplace=True)

#trigC_data = pd.read_csv("./ASIM/trigC.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
#trigC = pd.DataFrame(trigC_data)
#trigC.drop(columns=["######"], inplace=True)
#trigC.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time" ,"Corr":"corr"}, inplace=True)

#trigC_21_data = pd.read_csv("./ASIM/trigC_2021.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
#trigC_21 = pd.DataFrame(trigC_21_data)
#trigC_21.drop(columns=["######"], inplace=True)
#trigC_21.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time" }, inplace=True)

#trigM_data = pd.read_csv("./ASIM/trigM.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
#trigM = pd.DataFrame(trigM_data)
#trigM.drop(columns=["######"], inplace=True)
#trigM.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time", "Corr":"corr" }, inplace=True)

#trigM_21_data = pd.read_csv("./ASIM/trigM_2021.txt", sep = "\s+|\t+|\s+\t+|\t+\s+", engine="python")
#trigM_21 = pd.DataFrame(trigM_21_data)
#trigM_21.drop(columns=["######"], inplace=True)
#trigM_21.rename(columns={"yyyy-MMM-dd":"date","HH:mm:ss.SSSSSS": "time" }, inplace=True)

In [ ]:
#IPN datetime list
ipn_dt_temp = []
ipn_dict = ipn.to_dict("records")
for row in ipn_dict:
    datetime_str = row["time"]
    datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    ipn_dt_temp.append(datetime_obj)
    
ipn_dt = np.asarray(ipn_dt_temp)

## Vectorization and ASIM datetime correction

In [ ]:
# Function for correcting date and time in ASIM data. Returning matrix that contains datetime objects
# Method for correcting time
# 1. Retrive the time from time column
# 2. Isolate the microsecond time from that and cast it to an int
# 3. Retrive the correction time from Corr column and cast it to an int
# 4. Subtract correction time from time and cast it to an string
# 5. Insert the corrected time

# PROBLEM: ONLY ONE DATETIME IS ADDED

def corr_dt(dfs):  # Correcting times from ASIM data

    """   This function corrects the time from ASIM data.
    The correction is done by subtracting the correction from the original time.
    The correction is given as a string.
    The function takes a list of dataframes as input.
    The function returns a list of arrays containing the corrected datetime objects.
    
    Parameters
    ----------
    dfs : list of dataframes
        The dataframes containing the data.
    
    Returns
    -------
    trig_dt : list of arrays
        The corrected datetime objects.
    
    Raises
    ------
    ValueError
        If the lists are not the same length.
    """
    trig_dt = []
    
    for df in dfs:
        temp_dt = []
        # Vectorization of columns
        date = df["date"].values  # date given as string.
        time = df["time"].values  # time given as string
        corr = df["corr"].values  # correction given as string
        try:
            if len(date) and len(time) != len(corr):
                raise ValueError
        except:
            raise ValueError("Lists are not the same length")
        else:
            for i in np.arange(0, len(corr)):  # Iterating over the vectors
                if corr[i] == "--------":  # No correction needed. Appending the datetime object
                    date_str = date[i]
                    time_str = time[i]
                    org_dt = datetime.strptime(
                        date_str + " " + time_str, "%Y-%b-%d %H:%M:%S.%f")
                    temp_dt.append(org_dt)
                    
                elif corr[i][0] == "-":  # If it's a "-" in front; correction is added
                    # formatting the datetime object
                    date_str = date[i]
                    time_str = time[i]
                    org_dt = datetime.strptime(
                        date_str + " " + time_str, "%Y-%b-%d %H:%M:%S.%f")  # Original datetime

                    micro_corr = int(corr[0][1:])

                    # new corrected datetime. Timedelta ccounts for changes in seconds also
                    new_dt = org_dt + timedelta(microseconds=micro_corr)
                    temp_dt.append(new_dt)
                else:
                    date_str = date[i]
                    time_str = time[i]
                    org_dt = datetime.strptime(
                        date_str + " " + time_str, "%Y-%b-%d %H:%M:%S.%f")  # Original datetime

                    micro_corr = int(corr[0][1:])

                    # new corrected datetime. Timedelta ccounts for changes in seconds also
                    new_dt = org_dt - timedelta(microseconds=micro_corr)
                    temp_dt.append(new_dt)
                    
            trig_dt.append(temp_dt)

    trig_dt = np.asarray(trig_dt)
    return trig_dt

In [ ]:
#Callig corr_dt with a list containing the dataframes from ASIM
trig_dt = corr_dt([trigB])

## Algorithm for match between ASIM and IPN

In [ ]:
# Storing matches in a dictionary
# Need to speed up this algorithm. Because it's swallowing too much RAM


"""def search(ipn_dt, trig_dt, diff = 10):
    """This function takes in two arrays of datetime objects: one for IPN triggers and one for the triggers of choice.
    It returns a dictionary of lists of tuples. The dictionary contains the indices of the matching triggers for each
    type of trigger. The tuples contain the indices of the IPN and the trigger of choice.
    
    Parameters
    ----------
    ipn_dt : list of datetime objects
        The list of IPN triggers
    trig_dt : list of datetime objects: trig_dt = [trigB [<datetime object ...>], trigC[<datetime object ..>] ...]
        The list of triggers of choice
        
    Returns
    -------
    matches_dict : dictionary
        A dictionary of lists of tuples. The dictionary contains the indices of the matching triggers for each
        type of trigger. The tuples contain the indices of the IPN and the trigger of choice."""
    
    
    matches_dict = {"trigB": [], "trigC": [], "trigM": []}
    #Maybe use list comprehention: newlist = [x for x in fruits if "a" in x]
    
    for trig, col in enumerate(trig_dt):

        for index_trig, trig_time in enumerate(col):
            try:
                if trig == 0: #Maybe this can be modified?
                    for index_ipn, ipn_time in enumerate(ipn_dt):
                        t_delta = ipn_time - trig_time
                        if t_delta.total_seconds() <= diff:
                            matches_dict["trigB"].append((index_ipn, index_trig))
                        else:
                            continue

                elif trig == 1:
                    for index_ipn, ipn_time in enumerate(ipn_dt):
                        t_delta = ipn_time - trig_time
                        if t_delta.total_seconds() <= diff:
                            matches_dict["trigC"].append((index_ipn, index_trig))
                        else:
                            continue

                elif trig == 2:
                    for index_ipn, ipn_time in enumerate(ipn_dt):
                        t_delta = ipn_time - trig_time
                        if t_delta.total_seconds() <= diff:
                            matches_dict["trigM"].append((index_ipn, index_trig))
                        else:
                            continue

            except ValueError:
                print("Length of input trigger list is not correct")
    return matches_dict"""

In [ ]:
matches_dict = search(ipn_dt,trig_dt)

### Graveyard